In [1]:
import os
import h5py
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers
from transformers import ViTFeatureExtractor, TFViTModel,ViTImageProcessor
from PIL import Image
from io import BytesIO

2024-07-26 09:42:45.064068: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-26 09:42:45.064187: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-26 09:42:45.209536: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Load data

In [2]:
"""2024 ISIC Challenge primary prize scoring metric

Given a list of binary labels, an associated list of prediction 
scores ranging from [0,1], this function produces, as a single value, 
the partial area under the receiver operating characteristic (pAUC) 
above a given true positive rate (TPR).
https://en.wikipedia.org/wiki/Partial_Area_Under_the_ROC_Curve.

(c) 2024 Nicholas R Kurtansky, MSKCC
"""


class ParticipantVisibleError(Exception):
    pass


def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str, min_tpr: float=0.80) -> float:
    '''
    2024 ISIC Challenge metric: pAUC
    
    Given a solution file and submission file, this function returns the
    the partial area under the receiver operating characteristic (pAUC) 
    above a given true positive rate (TPR) = 0.80.
    https://en.wikipedia.org/wiki/Partial_Area_Under_the_ROC_Curve.
    
    (c) 2024 Nicholas R Kurtansky, MSKCC

    Args:
        solution: ground truth pd.DataFrame of 1s and 0s
        submission: solution dataframe of predictions of scores ranging [0, 1]

    Returns:
        Float value range [0, max_fpr]
    '''

    del solution[row_id_column_name]
    del submission[row_id_column_name]

    # check submission is numeric
    if not pd.api.types.is_numeric_dtype(submission.values):
        raise ParticipantVisibleError('Submission target column must be numeric')

    # rescale the target. set 0s to 1s and 1s to 0s (since sklearn only has max_fpr)
    v_gt = abs(np.asarray(solution.values)-1)
    
    # flip the submissions to their compliments
    v_pred = -1.0*np.asarray(submission.values)

    max_fpr = abs(1-min_tpr)

    # using sklearn.metric functions: (1) roc_curve and (2) auc
    fpr, tpr, _ = roc_curve(v_gt, v_pred, sample_weight=None)
    if max_fpr is None or max_fpr == 1:
        return auc(fpr, tpr)
    if max_fpr <= 0 or max_fpr > 1:
        raise ValueError("Expected min_tpr in range [0, 1), got: %r" % min_tpr)
        
    # Add a single point at max_fpr by linear interpolation
    stop = np.searchsorted(fpr, max_fpr, "right")
    x_interp = [fpr[stop - 1], fpr[stop]]
    y_interp = [tpr[stop - 1], tpr[stop]]
    tpr = np.append(tpr[:stop], np.interp(max_fpr, x_interp, y_interp))
    fpr = np.append(fpr[:stop], max_fpr)
    partial_auc = auc(fpr, tpr)

    #     # Equivalent code that uses sklearn's roc_auc_score
    #     v_gt = abs(np.asarray(solution.values)-1)
    #     v_pred = np.array([1.0 - x for x in submission.values])
    #     max_fpr = abs(1-min_tpr)
    #     partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
    #     # change scale from [0.5, 1.0] to [0.5 * max_fpr**2, max_fpr]
    #     # https://math.stackexchange.com/questions/914823/shift-numbers-into-a-different-range
    #     partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)
    
    return(partial_auc)

In [3]:

# Define the ISICDataset class
class ISICDataset(tf.keras.utils.Sequence):
    def __init__(self, hdf5_file, isic_ids, targets=None, feature_extractor=None, batch_size=32):
        self.hdf5_file = hdf5_file
        self.isic_ids = isic_ids
        self.targets = targets
        self.feature_extractor = feature_extractor
        self.batch_size = batch_size
        
    def __len__(self):
        return len(self.isic_ids)
    
    def __getitem__(self, idx):
        isic_id = str(self.isic_ids[idx])
        image = Image.open(BytesIO(self.hdf5_file[isic_id][()]))
        image = image.convert("RGB")
        image = np.array(image)
        if self.feature_extractor:
            image = self.feature_extractor(images=image, return_tensors="tf").pixel_values[0]
        if self.targets is not None:
            target = self.targets[idx]
            return image, target
        else:
            return image
    
    def _load_image(self, isic_id):
        isic_id = str(isic_id)
        image = Image.open(BytesIO(self.hdf5_file[isic_id][()]))
        image = image.convert("RGB")
        image = np.array(image)
        if self.feature_extractor:
            image = self.feature_extractor(images=image, return_tensors="tf").pixel_values[0]
        return image

    def _generator(self):
        for idx in range(len(self.isic_ids)):
            yield self._load_image(self.isic_ids[idx]), self.targets[idx]

    def to_tf_dataset(self):
        tf_dataset = tf.data.Dataset.from_generator(self._generator, 
                                                    output_signature=(tf.TensorSpec(shape=(3, 224, 224), dtype=tf.float32),
                                                                      tf.TensorSpec(shape=(), dtype=tf.int64)))
        tf_dataset = tf_dataset.batch(self.batch_size).prefetch(buffer_size=tf.data.AUTOTUNE).repeat()
        return tf_dataset

In [4]:
# Load data
data_path = '/kaggle/input/isic-2024-challenge/'

path_train_hdf5 = os.path.join(data_path, 'train-image.hdf5')
path_test_hdf5 = os.path.join(data_path, 'test-image.hdf5')

path_train_meta = data_path + 'train-metadata.csv'
path_test_meta = data_path + 'test-metadata.csv'


In [5]:
train_hdf5 = h5py.File(path_train_hdf5, 'r')
test_hdf5 = h5py.File(path_test_hdf5, 'r')

In [6]:
train_meta = pd.read_csv(path_train_meta)
test_meta = pd.read_csv(path_test_meta)

/tmp/ipykernel_18/938936479.py:1: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  train_meta = pd.read_csv(path_train_meta)


In [7]:
# read in the isic ids and target values
train_isic_ids = train_meta['isic_id'].values
train_isic_ids = train_meta[train_meta['lesion_id'].notnull()]['isic_id'].values

test_isic_ids = test_meta['isic_id'].values

train_targets = train_meta[train_meta['lesion_id'].notnull()]['target'].values

In [8]:
total_size = len(train_targets)
indices = np.arange(total_size)

train_size = int(total_size * 0.8)
val_size = total_size - train_size

train_indices, val_indices = train_test_split(indices, test_size=val_size, train_size=train_size)


In [9]:
# Laad de feature extractor
# model_name = 'google/vit-base-patch16-224'
# feature_extractor = ViTFeatureExtractor.from_pretrained(model_name)
# # Laad het ViT-model
# vit_model = TFViTModel.from_pretrained(model_name)


# # # Sla de feature extractor en het model lokaal op
# feature_extractor.save_pretrained('./kaggle/tmp/vit-base-patch16-224-finetuned')
# vit_model.save_pretrained('./kaggle/tmp/vit-base-patch16-224-finetuned')


# Laad de feature extractor en het model lokaal
feature_extractor = ViTFeatureExtractor.from_pretrained('/kaggle/input/pre_trained_vit16/tensorflow2/default/1')
vit_model = TFViTModel.from_pretrained('/kaggle/input/pre_trained_vit16/tensorflow2/default/1')



/opt/conda/lib/python3.10/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
All model checkpoint layers were used when initializing TFViTModel.

All the layers of TFViTModel were initialized from the model checkpoint at /kaggle/input/pre_trained_vit16/tensorflow2/default/1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTModel for predictions without further training.


In [10]:

# # # Maak de train en val datasets
# train_dataset = ISICDataset(train_hdf5, train_isic_ids[train_indices], targets=train_targets[train_indices], feature_extractor=feature_extractor)
# val_dataset = ISICDataset(train_hdf5, train_isic_ids[val_indices], targets=train_targets[val_indices], feature_extractor=feature_extractor)


In [11]:
# # # Maak TensorFlow datasets
# tf_train_dataset = train_dataset.to_tf_dataset()
# tf_val_dataset = val_dataset.to_tf_dataset()

In [12]:
# # # Combineer de datasets
# combined_isic_ids = np.concatenate([train_isic_ids[train_indices], train_isic_ids[val_indices]])
# combined_targets = np.concatenate([train_targets[train_indices], train_targets[val_indices]])

# combined_dataset = ISICDataset(train_hdf5, combined_isic_ids, targets=combined_targets, feature_extractor=feature_extractor)
# tf_combined_dataset = combined_dataset.to_tf_dataset()


In [13]:
# detect and init the TPU
# tpu = tf.distribute.cluster_resolver.TPUClusterResolver()

# # instantiate a distribution strategy
# tf.tpu.experimental.initialize_tpu_system(tpu)
# tpu_strategy = tf.distribute.TPUStrategy(tpu)

In [14]:
# # Definieer de CustomViTModel class
# class CustomViTModel(tf.keras.Model):
#     def __init__(self, backbone, num_classes, **kwargs):
#         super(CustomViTModel, self).__init__(**kwargs)
#         self.backbone = backbone
#         self.dense = layers.Dense(num_classes, activation='softmax')

#     def call(self, inputs):
#         outputs = self.backbone(inputs).last_hidden_state[:, 0, :]
#         outputs = self.dense(outputs)
#         return outputs

In [15]:
# Maak het CustomViTModel
# num_classes = len(np.unique(train_targets))
# custom_vit_model = CustomViTModel(vit_model, num_classes)

In [16]:
# # Compileer het model met een gebruikelijke loss-functie
# with tpu_strategy.scope():
#     custom_vit_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
#                              loss='sparse_categorical_crossentropy',
#                              metrics=['accuracy'])

In [17]:
# steps_per_epoch = len(combined_dataset) // combined_dataset.batch_size

In [18]:
# Train het model met de gecombineerde dataset
# with tpu_strategy.scope():
#     history = custom_vit_model.fit(tf_combined_dataset, epochs=10, steps_per_epoch=steps_per_epoch)

In [19]:
#Sla de modelgewichten op
# with tpu_strategy.scope():
#     custom_vit_model.save_weights('./kaggle/tmp/custom_vit_model_combi.weights.h5')

# Sla de feature extractor en het model lokaal op
# feature_extractor.save_pretrained('./vit-base-patch16-224')
# vit_model.save_pretrained('./vit-base-patch16-224')

In [20]:

# # Verkrijg de voorspellingen van het model
# predictions = custom_vit_model.predict(val_images)
# predicted_probabilities = tf.reduce_max(predictions, axis=1)

# # Binarize de test labels voor ROC berekening
# val_labels_binary = (val_labels == 1).numpy().astype(int)


In [21]:
# # Bereken de ROC-curve en de pAUC
# fpr, tpr, thresholds = roc_curve(val_labels_binary, predicted_probabilities)
# partial_auc = auc(fpr, tpr)

# print(f'Partial AUC: {partial_auc}')

In [22]:
import tensorflow as tf
from transformers import ViTFeatureExtractor, TFViTModel
import h5py
import pandas as pd
from PIL import Image
from io import BytesIO
import numpy as np

# Definieer de CustomViTModel class
class CustomViTModel(tf.keras.Model):
    def __init__(self, backbone, num_classes, **kwargs):
        super(CustomViTModel, self).__init__(**kwargs)
        self.backbone = backbone
        self.dense = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        outputs = self.backbone(inputs).last_hidden_state[:, 0, :]
        outputs = self.dense(outputs)
        return outputs

# Registreren van de custom class
tf.keras.utils.get_custom_objects()['CustomViTModel'] = CustomViTModel

# Laad de feature extractor en het model lokaal
feature_extractor = ViTFeatureExtractor.from_pretrained('/kaggle/input/pre_trained_vit16/tensorflow2/default/1', local_files_only=True)
vit_model = TFViTModel.from_pretrained('/kaggle/input/pre_trained_vit16/tensorflow2/default/1', local_files_only=True)

num_classes = len(np.unique(train_targets))
custom_vit_model = CustomViTModel(vit_model, num_classes)
custom_vit_model.load_weights('/kaggle/input/pre_trained_vit16/tensorflow2/default/2/custom_vit_model_combi.weights.h5')

# Functie om testset voor te bereiden en te voorspellen
def prepare_test_dataset(test_hdf5, test_isic_ids, feature_extractor, batch_size=64):
    def _generator():
        for isic_id in test_isic_ids:
            isic_id = str(isic_id)
            image = Image.open(BytesIO(test_hdf5[isic_id][()]))
            image = image.convert("RGB")
            image = np.array(image)
            image = feature_extractor(images=image, return_tensors="tf").pixel_values[0]
            yield image
    
    tf_test_dataset = tf.data.Dataset.from_generator(_generator, 
                                                     output_signature=tf.TensorSpec(shape=(3, 224, 224), dtype=tf.float32))
    tf_test_dataset = tf_test_dataset.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
    return tf_test_dataset

# Voorbereiden van de testset
data_path = '/kaggle/input/isic-2024-challenge/'
path_test_hdf5 = os.path.join(data_path, 'test-image.hdf5')
path_test_meta = data_path + 'test-metadata.csv'

test_hdf5 = h5py.File(path_test_hdf5, 'r')
test_meta = pd.read_csv(path_test_meta)
test_isic_ids = test_meta['isic_id'].values

tf_test_dataset = prepare_test_dataset(test_hdf5, test_isic_ids, feature_extractor)

# Schakel mixed precision training in
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')

# Genereer voorspellingen voor de testset
predictions = custom_vit_model.predict(tf_test_dataset)
predicted_probabilities = tf.reduce_max(predictions, axis=1).numpy()


# Maak de submission dataframe
submission = pd.DataFrame({'isic_id': test_isic_ids, 'target': predicted_probabilities})

# Sla de submission dataframe op als CSV
submission.to_csv('submission.csv', index=False)


All model checkpoint layers were used when initializing TFViTModel.

All the layers of TFViTModel were initialized from the model checkpoint at /kaggle/input/pre_trained_vit16/tensorflow2/default/1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTModel for predictions without further training.


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step


/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)
